In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

1. Import

In [2]:
df = pd.read_csv('data/bronze_cleaned.csv')

### 2. Feature Engineering

2.1 Total

In [3]:
#Nouvelle colonne basée sur la multiplication du prix d'un article par sa quantité commandée
df['Total'] = df.Quantity * df.Price

    ^ coût total d'une seule ligne

2.2 Order Quantity

In [4]:
#On compare les lignes ayant le même numéro de facture et on additionne les quantités
df['Order_Quantity'] = df.groupby('Invoice')['Quantity'].transform('sum')
df[['Order_Quantity']].sample(1)

,Order_Quantity
444228,1082


    ^ Nombre total d'article sur une seule commande

2.3 Order Total

In [5]:
#On additionnne tous les totaux d'une même commande
df['Order_Total'] = df.groupby('Invoice')['Total'].transform('sum')
df[['Order_Total']].sample(1)

,Order_Total
793939,208.39


    ^ Coût total d'une même commande

2.4 Cancelled

In [6]:
#La documentation du dataset indique que les Invoice commençant par 'C' sont des annulations
#Verifie si le numéro de facture commence par un C et attribue un booléen en fonction
df['Cancelled'] = df.Invoice.apply(lambda x : True if x.startswith('C') else False)
df[df['Cancelled'] == True][['Invoice','Cancelled']].sample(1)

,Invoice,Cancelled
548287,C540792,True


    ^ Indique le statut de la commande (valide/retour)

2.5 Order count

In [7]:
#Compte le nombre d'Invoice unique qui n'est pas une annulation par client et lui attribut cette valeur
df['Order_count'] = df.groupby('Customer ID')['Invoice'].transform('nunique')
df['Order_count'] = df['Order_count'].fillna(0)
df[['Order_count']].head(1)

,Order_count
0,10


    ^ nombre total de commandes pour le client

In [8]:
# Compte le nombre d'invoice unique avec le masque "commande annulée = vrai"
df['Cancelled_order_count'] = df[df.Cancelled == True].groupby('Customer ID')['Invoice'].transform('nunique')
df['Cancelled_order_count'] = df['Cancelled_order_count'].fillna(0)
df[df.Cancelled == True][['Cancelled_order_count']].head(1)

,Cancelled_order_count
178,3.0


    ^ nombre de commandes annulées du client

In [9]:
# Fais le produit en croix entre le nombre de commandes annulées et le nombre de commandes totales
df['Percent_cancelled'] = (df['Cancelled_order_count'] * 100) / df['Order_count']
df[df.Cancelled == True][['Percent_cancelled']].head(1)

,Percent_cancelled
178,30.0


    ^ pourcentage de commandes annulées du client

2.6 Average Quantity

In [10]:
df['Average_Quantity'] = round(df.groupby('Customer ID')['Quantity'].transform('mean'),2)
df[['Average_Quantity']].head(1)

,Average_Quantity
0,9.08


    ^ quantité moyenne commandée pour 1 item

In [11]:
df['Average_Price'] = round(df.groupby('Customer ID')['Price'].transform('mean'),2)
df[['Average_Price']].head(1)

,Average_Price
0,12.41


    ^ Prix moyen des articles commandés par le client

In [12]:
df['Average_Basket'] = round(df.groupby(['Customer ID', 'Invoice'])['Total'].transform('mean'),2)
df[['Average_Basket']].head(1)

,Average_Basket
0,63.16


    ^ panier moyen du client

In [13]:
df['StockCode_count'] = df.groupby('Customer ID')['StockCode'].transform('nunique')
df[['StockCode_count']].head(1)

,StockCode_count
0,51


    ^ Nombre d'item differents commandés par le client toutes commandes confondues

In [14]:
df.InvoiceDate.isna().value_counts()

False    1033036
Name: InvoiceDate, dtype: int64

In [15]:
# Convertir la colonne "InvoiceDate" en format datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Calculer la date la plus récente
latest_date = df['InvoiceDate'].max()
# Calculer la récence (R)
df['Recency'] = (latest_date - df['InvoiceDate']).dt.days
df[['Recency']].head(1)

,Recency
0,738


    ^ Nombre de jours depuis la dernière commande

Bonus : Flags (pour les visualisations)

In [16]:
# Dictionnaire Pays / Drapeaux
country_flags = {
    'United Kingdom': '🇬🇧', 'France': '🇫🇷', 'USA': '🇺🇸', 'Belgium': '🇧🇪', 'Australia': '🇦🇺','EIRE': '🇮🇪', 'Germany': '🇩🇪', 'Portugal': '🇵🇹', 'Japan': '🇯🇵', 'Denmark': '🇩🇰','Nigeria': '🇳🇬', 'Netherlands': '🇳🇱', 'Poland': '🇵🇱', 'Spain': '🇪🇸', 'Channel Islands': '🇬🇬','Italy': '🇮🇹', 'Cyprus': '🇨🇾', 'Greece': '🇬🇷', 'Norway': '🇳🇴', 'Austria': '🇦🇹','Sweden': '🇸🇪', 'United Arab Emirates': '🇦🇪', 'Finland': '🇫🇮', 'Switzerland': '🇨🇭','Unspecified': '🏳️', 'Malta': '🇲🇹', 'Bahrain': '🇧🇭', 'RSA': '🇿🇦', 'Bermuda': '🇧🇲','Hong Kong': '🇭🇰', 'Singapore': '🇸🇬', 'Thailand': '🇹🇭', 'Israel': '🇮🇱', 'Lithuania': '🇱🇹','West Indies': '🌴', 'Lebanon': '🇱🇧', 'Korea': '🇰🇷', 'Brazil': '🇧🇷', 'Canada': '🇨🇦','Iceland': '🇮🇸'
}

In [17]:
#Mapping du drapeau via la colonne pays
df['Flag'] = df['Country'].map(country_flags)
df[['Flag']].head(1)

,Flag
0,🇬🇧


In [18]:
# Diviser le DataFrame en deux parties en fonction de l'année
mask_2009_2010 = (df['InvoiceDate'].dt.year < 2011)
silver_I = df[mask_2009_2010]
silver_II = df[~mask_2009_2010]

# Enregistrer les deux parties dans des fichiers distincts
silver_I.to_csv('data/silver_2009_2010.csv', index=False)
silver_II.to_csv('data/silver_2011.csv', index=False)
